In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import nbimporter
import Weather

Importing Jupyter notebook from Weather.ipynb


# 太陽位置を計算するモジュール

## 1. Functions  

任意の時刻・緯度・経度における太陽高度角・太陽方位角を計算する。

### 入力値  

$ Y $ ：西暦年（year）  
$ D $ ：1月1日を初日（$D=1$）とする計算対象日の年通算日 (day)  
$ T_m $ ：標準時 (h)  
$ \Phi $ ：計算対象地点の緯度 (°)  
$ L $ ：計算対象地点の経度 (°)  

### 出力値

$ h $ ：太陽高度角 (°)  
$ A $ ：太陽方位角 (°)  

### 計算方法

#### データ構造と値の範囲

$Y$・$D$・$T_m$・$\Phi$・$L$の組み合わせを1つの変数（タプル）で表し"sp_input"とする。  
$h$は0°以上、90°以下の値をとる（地平：0°、天頂：90°）。  
$A$は-180°以上、180°以下の値をとる（東：-90°、南：0°、西：90°）。

In [2]:
sp_input = namedtuple('sp_input','sp_year sp_nday sp_tm sp_lat sp_lon')
#sp_year：西暦年（year）  
#sp_nday：1月1日を初日（$D=1$）とする計算対象日の年通算日 (day)  
#sp_tm：標準時 (h)  
#sp_lat：計算対象地点の緯度 (°)  
#sp_lon：計算対象地点の経度 (°)  

#### 太陽高度角及び太陽方位角の計算

（参考：拡張アメダス気象データ1981-2000解説書 8.1　太陽位置の計算）  

太陽高度角$ h $、太陽方位角$ A $の計算方法を以下に示す。  

$ \begin{align}\sin h = \sin\Phi ･ \sin \delta + \cos \Phi ･ \cos \delta ･ \cos t\end{align}$ ････････････････ (3.1-a)  

$ \begin{align}\sin A = \frac{\cos\delta ･ \sin t}{\cos h}\end{align}$ ････････････････････････････････････ (3.1-b)  

$ \begin{align}\cos A = \frac{\sin h ･ \sin\Phi - \sin\delta}{\cos h ･ \cos\phi}\end{align}$ ････････････････････････････ (3.1-c)  

$ \begin{align}t = 15 ･ (T_m - 12) + (L - L_0) + E_t\end{align}$ ･･････････････････････ (3.1-d)  

$ \begin{align}\sin\delta = \cos(v + \epsilon) ･ \sin\delta_0\end{align}$ ･･･････････････････････････････ (3.1-e)  

$ \begin{align}E_t = (M - v) - \tan^{-1}\biggl[\frac{0.043 ･ \sin 2(v + \epsilon)}{1 - 0.043 ･ \cos 2(v + \epsilon)}\biggr]\end{align}$ ･････････ (3.1-f)  

$ \begin{align}v = M + 1.914 ･ \sin M + 0.02 ･ \sin(2M)\end{align}$ ･･････････････････ (3.1-g)  

$ \begin{align}\epsilon = 12.3901 + 0.0172 ･ \biggl(n + \frac{M}{360}\biggr)\end{align}$ ･････････････････････ (3.1-h)  

$ \begin{align}M = \frac{360 ･ (D - d_0)}{D_{ay}} \end{align}$ ･････････････････････････････････････ (3.1-i)  

$ \begin{align}d_0 = 3.71 + 0.2596 ･ n - INT\biggl[\frac{n+3}{4}\biggr]\end{align}$ ･･･････････････････ (3.1-j)  

$ \begin{align}n = Y - 1968\end{align}$ ･･････････････････････････････････････････ (3.1-k)  

ここで、  

$ \delta $ ：赤緯 (°)  
$ t $ ：時角 (°)  
$ L_0 $ ：標準時の地点の経度（=135.0（日本の場合）） (°)  
$ E_t $ ：均時差 (°)  
$ v $ ：真近点離角 (°)  
$ \epsilon $ ：近日点と冬至点の角度 (°)  
$ \delta_0 $ ：北半球の冬至の日赤緯（= -23.4393） (°)  
$ M $ ：平均近点離角 (°)  
$ n $ ：1968年との年差 (year)  
$ d_0 $ ：平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）(day)  
$ D_{ay} $ ：近点年（近日点基準の公転周期日数）（= 365.2596） (day)  
$ INT[] $ ：小数点以下切り捨て  
$ Y $ ：西暦年 (year)  

である。  

In [3]:
#太陽位置の計算
#太陽高度角、太陽方位角
def solar_position(L):
    
    sp_n = L.sp_year - 1968.
    sp_d0 = 3.71 + 0.2596 * sp_n - int((sp_n + 3.) / 4.)
    sp_Day = 365.2596
    sp_M = 360. * (L.sp_nday - sp_d0) / sp_Day
    sp_eps = 12.3901 + 0.0172 * (sp_n + sp_M / 360.)
    sp_v = sp_M + 1.914 * math.sin(math.radians(sp_M)) + 0.02 * math.sin(math.radians(2. * sp_M))
    sp_veps = math.radians(sp_v + sp_eps)
    sp_et = (sp_M - sp_v) - math.degrees(math.atan(0.043 * math.sin(2. * sp_veps) / (1. - 0.043 * math.cos(2. * sp_veps))))
    sp_dlt0 = math.radians(-23.4393)
    sp_sindlt = math.cos(sp_veps) * math.sin(sp_dlt0)
    sp_cosdlt = (abs(1. - sp_sindlt ** 2.)) ** 0.5
    sp_lons = 135.
    sp_t = 15. * (L.sp_tm - 12.) + (L.sp_lon - sp_lons) + sp_et
    sp_latrad = math.radians(L.sp_lat)
    sp_trad = math.radians(sp_t)
    sp_sinh = math.sin(sp_latrad) * sp_sindlt + math.cos(sp_latrad) * sp_cosdlt * math.cos(sp_trad)
    sp_cosh = (1. - sp_sinh ** 2.) ** 0.5
    sp_sinA = sp_cosdlt * math.sin(sp_trad) / sp_cosh
    sp_cosA = (sp_sinh * math.sin(sp_latrad) - sp_sindlt) / (sp_cosh * math.cos(sp_latrad))
    
    r_sp_h = 0.
    r_sp_A = 0.
    if sp_sinh > 0.:
        r_sp_h = math.degrees(math.asin(sp_sinh))
        if sp_sinA > 0.:
            r_sp_A = 90. - math.degrees(math.atan(sp_cosA / sp_sinA))
        elif sp_sinA < 0.:
            r_sp_A = - 90. - math.degrees(math.atan(sp_cosA / sp_sinA))
        elif sp_sinA == 0.:
            r_sp_A = 0.
        
    return r_sp_h, r_sp_A

#### Example

In [1]:
location = '3639999'               #東京ID
year = 1995                        #西暦年
lat = 35. + 41.2 / 60.             #計算対象地点の緯度 (°)  
lon = 139. + 45.9 / 60.            #計算対象地点の経度 (°)  

sp_result = np.empty((5,8760))

for i in range(0, 8760):
    sp_in = sp_input(year, int((i + 1.) / 24.) + 1., ((i + 1.) / 24. - int((i + 1.) / 24.)) * 24., lat, lon)
    sp_result[0][i] = i + 1.                                           #年通算時間
    sp_result[1][i] = int((i + 1.) / 24.) + 1.                         #年通算日
    sp_result[2][i] = ((i + 1.) / 24. - int((i + 1.) / 24.)) * 24.     #時刻
    sp_result[3][i] = solar_position(sp_in)[0]                         #太陽高度角
    sp_result[4][i] = solar_position(sp_in)[1]                         #太陽方位角
    
#np.savetxt(location + '_out_sp.csv', np.transpose(sp_result), fmt='%.10f', delimiter=',')

#確認用気象データ読込
h_ea = Weather.weather_confirm_input(location)[5]                              #太陽高度角
A_ea = Weather.weather_confirm_input(location)[6]                              #太陽方位角

#グラフの描画
graph_num = 2
ax = np.empty(graph_num, dtype = np.object)
fig = plt.figure(figsize = (15, 5 * ((graph_num + 1) // 2)))
plt.style.use('ggplot')

gpc = 'C0'
gxd = [[h_ea[0:8760], [0,90]], [A_ea[0:8760], [-120,120]]]
gyd = [[sp_result[3][0:8760], [0,90]], [sp_result[4][0:8760], [-120,120]]]
gxl = ['h(eadata) (°)','A(eadata) (°)']
gyl = ['h(calc) (°)', 'A(calc) (°)']
gxt = [[10 * i for i in range(10)], [30 * (i - 4) for i in range(9)]]
gyt = [[10 * i for i in range(10)], [30 * (i - 4) for i in range(9)]]

for i in range(graph_num):   
    ax[i] = fig.add_subplot((graph_num + 1) // 2, 2, i + 1)
    ax[i].scatter(gxd[i][0], gyd[i][0], color = gpc)
    ax[i].plot(gxd[i][1], gyd[i][1], color = (0,0,0))
    ax[i].set_xticks(gxt[i])
    ax[i].set_yticks(gyt[i])
    ax[i].set_xlabel(gxl[i], fontsize = 12)
    ax[i].set_ylabel(gyl[i], fontsize = 12)

plt.show()

NameError: name 'np' is not defined